### Working Demand Matrix Generation using Grid2Demand
This notebook demonstrates how to use grid2demand to build a travel demand matrix for any urban area.

In [1]:
# Setup and Imports
import os

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox

import grid2demand as gd
import osm2gmns as og

import warnings
warnings.filterwarnings('ignore')

# Ensure Directory Location and Existence

# Create output directory
output_dir = 'data'
os.makedirs(output_dir, exist_ok=True)
osm_filepath = f"{output_dir}/map.osm"

In [2]:
# Select, Download, and Save OSM Network Data for Grid2Demand Using osm2gmns Library

# OSM Area ids
# search at: nominatim.openstreetmap.org
brooklyn_id = 369518
antwerp_id = 53114
brussels_id = 2404020
ghent_id = 897671
lviv_id = 2032280

# Download directly from OSM
print("Downloading OSM network...")
og.downloadOSMData(brooklyn_id, osm_filepath)

# Open OSM, convert to GMNS, and clean as desired
print("Loading and converting OSM network to GMNS...")
net = og.getNetFromFile(osm_filepath, mode_types=['auto', 'railway', 'walk', 'bike'], POI=True)
og.fillLinkAttributesWithDefaultValues(net)
#og.consolidateComplexIntersections(net, auto_identify=True)

# Save to CSV files in GMNS format
og.outputNetToCSV(net, output_dir)

print(f"  Nodes: {net.number_of_nodes:,}")
print(f"  Links: {net.number_of_links:,}")


: 

In [ ]:
# Initialize, Load Grid2Demand Network

# Create a GRID2DEMAND object (newer API)
net = gd.GRID2DEMAND(
    input_dir=output_dir,
    mode_type='auto'
    )

# Load the network files
net.load_network()

# Generate zone.csv from node by specifying number of x blocks and y blocks
#net.net2grid(num_x_blocks=15, num_y_blocks=15)
net.net2grid(cell_width=400, cell_height=400, unit="meter")

# Load zone from zone.csv
net.taz2zone()

# Map zones with nodes and poi, viseversa
net.map_zone_node_poi()

# Calculate zone-to-zone distance matrix
net.calc_zone_od_distance(pct=1.0)

# RUN Gravity model and save results
net.run_gravity_model()

net.save_results_to_csv(zone_od_dist_matrix=True, demand_od_matrix=True, overwrite_file=True)



In [ ]:
# Plot Results
from plot_brooklyn_results import plot_results
plot_results(output_dir)